In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets( 'tensorflow/models', one_hot=True )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorflow/models/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorflow/models/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tensorflow/models/t10k-images-idx3-ubyte.gz
Extracting tensorflow/models/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
num_epochs = 20
n_features = 28
n_hidden = 32
n_classes = 10
lr = 0.01

In [4]:
with tf.variable_scope('lstm_cell'):
    ## Placeholder for input vector with shape[batch, seq, embeddings]
    x = tf.placeholder(dtype=tf.float32,shape=[None,None,n_features])
    ## Placeholder for input vector with shape[batch, num_classes]
    y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])
    # Processing inputs to work with scan function
    # Process tensor of size [5,3,2] to [3,5,2]
    batch_input_ = tf.transpose(x, perm=[2, 0, 1])
    processed_input = tf.transpose(batch_input_)
    ## Hidden Vector Initially
    initial_hidden = x[:, 0, :]
    initial_hidden = tf.matmul(initial_hidden, tf.zeros([n_features, n_hidden]))
    initial_hidden = tf.stack([initial_hidden, initial_hidden])
    ## Forget Gate
    wf = tf.Variable(tf.random_normal([n_features,n_hidden]))
    uf = tf.Variable(tf.random_normal([n_hidden,n_hidden]))
    bf = tf.Variable(tf.random_normal([n_hidden]))
    ## Input Gate
    wi = tf.Variable(tf.random_normal([n_features,n_hidden]))
    ui = tf.Variable(tf.random_normal([n_hidden,n_hidden]))
    bi = tf.Variable(tf.random_normal([n_hidden]))
    ## New candidate Gate
    wc = tf.Variable(tf.random_normal([n_features,n_hidden]))
    uc = tf.Variable(tf.random_normal([n_hidden,n_hidden]))
    bc = tf.Variable(tf.random_normal([n_hidden]))
    ## Output Gate
    wog = tf.Variable(tf.random_normal([n_features,n_hidden]))
    uog = tf.Variable(tf.random_normal([n_hidden,n_hidden]))
    bog = tf.Variable(tf.random_normal([n_hidden]))
    ## Output
    wo = tf.Variable(tf.random_normal([n_hidden,n_classes]))
    bo = tf.Variable(tf.random_normal([n_classes]))
    

In [5]:
def graph_def(prev_hidden, x):
    previous_hidden_state, c_prev = tf.unstack(prev_hidden)
    i = tf.sigmoid(tf.matmul(x,wi) + tf.matmul(previous_hidden_state,ui) + bi)
    f = tf.sigmoid(tf.matmul(x,wf) + tf.matmul(previous_hidden_state,uf) + bf)
    o = tf.sigmoid(tf.matmul(x,wog) + tf.matmul(previous_hidden_state,uog) + bog)
    c_ = tf.nn.tanh(tf.matmul(x,wc) + tf.matmul(previous_hidden_state,uc) + bc)
    c = f*c_prev + i*c_
    current_hidden_state = o * tf.nn.tanh(c)
    return tf.stack([current_hidden_state,c])

In [6]:
def get_states():
        all_hidden_states = tf.scan(graph_def, processed_input, initializer=initial_hidden)
        all_hidden_states = all_hidden_states[:, 0, :, :]
        return all_hidden_states

In [7]:
def get_output(hidden_state):
        output = tf.nn.relu(tf.matmul(hidden_state, wo) + bo)
        return output

In [8]:
def get_outputs():
        all_hidden_states = get_states()
        all_outputs = tf.map_fn(get_output, all_hidden_states)
        return all_outputs

In [9]:
outputs = get_outputs()
prediction = tf.nn.softmax(outputs[-1])

In [10]:
cost = -tf.reduce_sum(y * tf.log(prediction))
saver = tf.train.Saver(max_to_keep=10)
summary = tf.summary.scalar('cost',cost)

In [11]:
train_writer = tf.summary.FileWriter('./logs/training_lstm')
train_writer.add_graph( tf.get_default_graph() )

In [12]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epochs):
        for _ in range(100):
            data_x,data_y = mnist.train.next_batch(32)
            data_x = data_x.reshape(-1,28,28)
            sess.run(optimizer,feed_dict={
                x:data_x,
                y:data_y
            })
            data_x,data_y = mnist.train.next_batch(256)
            data_x = data_x.reshape(-1,28,28)
            training_loss,train_summary = sess.run([cost,summary],feed_dict = {
                x:data_x,
                y:data_y
            })
            train_writer.add_summary(train_summary,epoch)
            test_loss=0.0
            for _ in range(50):
                data_x,data_y = mnist.test.next_batch(32)
                data_x = data_x.reshape(-1,28,28)
                test_loss += sess.run(cost, feed_dict = {
                    x:data_x,
                    y:data_y
                })
            test_loss = test_loss/50
        print('Epoch: '+str(epoch+1))
        print ( 'Training Loss is ' + str(training_loss) + '\nTesting loss is ' + str(test_loss) + '\n')
        saver.save(sess, './lstm_weights/epoch_' + str(epoch + 1) + '.ckpt')
        print (" Model Saved \n ")
        print (" ----------- \n")

Epoch: 1
Training Loss is 458.387
Testing loss is 56.66271049499512

 Model Saved 
 
 ----------- 

Epoch: 2
Training Loss is 435.32993
Testing loss is 54.946864471435546

 Model Saved 
 
 ----------- 

Epoch: 3
Training Loss is 427.4062
Testing loss is 52.06788185119629

 Model Saved 
 
 ----------- 

Epoch: 4
Training Loss is 392.66907
Testing loss is 49.62153457641602

 Model Saved 
 
 ----------- 

Epoch: 5
Training Loss is 376.0315
Testing loss is 45.02549324035645

 Model Saved 
 
 ----------- 

Epoch: 6
Training Loss is 377.192
Testing loss is 44.138297653198244

 Model Saved 
 
 ----------- 

Epoch: 7
Training Loss is 385.49652
Testing loss is 43.54977798461914

 Model Saved 
 
 ----------- 

Epoch: 8
Training Loss is 322.81958
Testing loss is 41.13496253967285

 Model Saved 
 
 ----------- 

Epoch: 9
Training Loss is 345.65494
Testing loss is 38.46438251495361

 Model Saved 
 
 ----------- 

Epoch: 10
Training Loss is 297.94382
Testing loss is 33.891560344696046

 Model Saved 